# Strategy  with RSI
The strategy will be created using the RSI indicator (detailed description in the next box).

### Content
* Explanation and computation of the RSI
* Do a strategy with the RSI
* Automate the strategy

In [ ]:
# import library
import numpy as np
import yfinance as yf
import ta
import warnings
warnings.filterwarnings("ignore")


In [ ]:
# The code here will allow you to switch your graphics to dark mode for those who choose to code in dark mode

import matplotlib.pyplot as plt
from matplotlib import cycler
colors = cycler('color',
                ['#669FEE', '#66EE91', '#9988DD',
                 '#EECC55', '#88BB44', '#FFBBBB'])
plt.rc('figure', facecolor='#313233')
plt.rc('axes', facecolor="#313233", edgecolor='none',
       axisbelow=True, grid=True, prop_cycle=colors,
       labelcolor='gray')
plt.rc('grid', color='474A4A', linestyle='solid')
plt.rc('xtick', color='gray')
plt.rc('ytick', direction='out', color='gray')
plt.rc('legend', facecolor="#313233", edgecolor="#313233")
plt.rc("text", color="#C9C9C9")

# Section 1: Explanation and computation of the RSI

### Explanation
The RSI (Relative Strength Index) will allow you to understand the strength of a trend and identify the end of a trend. The RSI value will vary from 0 to 100.

* If the RSI is around the 70 level, it is assumed that the instrument is overbought. This is a situation where prices have risen sharply, and the movement is likely to weaken.
* If the RSI is around the 30 level, this is a technical signal that the instrument may be oversold. In this case, it is a situation where prices have fallen sharply, and the movement may now lose strength.
* If the indicator oscillates horizontally around the 50 level, it means that the market is lacking a trend. The 50 level is the middle line that separates the bullish and bearish territories of the indicator...

It should be taken into account that the RSI is an indicator of the oscillator family, so it will work a little less well in markets with less volatility, such as the stock market.




In [ ]:
# Import the data
data_set = yf.download("GOOG")

In [ ]:
# Compute the rsi
data_set["rsi"] = ta.momentum.RSIIndicator(data_set["Adj Close"], window=14).rsi()
data_set

In [ ]:

# Plot the RSI with overbuy and oversell threshold

# Adapt the size of the graph
plt.figure(figsize=(15,6))

# View the RSI
plt.plot(data_set["rsi"].loc["2010"])

# View horizontal line for Overbuy threshold (RSI=70)
plt.axhline(70, color="#57CE95")

# View horizontal line for Oversell threshold (RSI=30)
plt.axhline(30, color="#CE5757")

# Put a title
plt.title("RSI with thresholds")

# Put a legend
plt.legend(["RSI", "Overbuy threshold", "Oversell threshold"])

# Show the graph
plt.show()

# Section 2: Do a strategy with the RSI

In [ ]:
# Plot the RSI with the Long buy zone and Short sell zone

# Adapt the size of the graph
plt.figure(figsize=(15,6))

# View the RSI
plt.plot(data_set["rsi"].loc["2010"])

# View horizontal line for Overbuy threshold (RSI=70)
plt.fill_between(data_set["rsi"].loc["2010"].index, 55,
                 70, 55 < 70, color="#57CE95",
                 alpha=0.5)

# View horizontal line for Oversell threshold (RSI=30)
plt.fill_between(data_set["rsi"].loc["2010"].index, 45,
                 30, 30 < 45, color="#CE5757",
                 alpha=0.5)
# Put a title
plt.title("RSI with zone of long buy and short sell")

# Put a legend
plt.legend(["RSI", "Long buy zone", "Short sell zone"])

# Show the graph
plt.show()

In [ ]:
# We define when the strategy must take a long buy or a short sell

# We need define the Open Long signal and Close Long signal

# We put threshold
overbuy = 70
neutral_buy = 55

# Put nan values for the signal long columns
data_set["signal_long"] = np.nan
data_set["yersteday_rsi"] = data_set["rsi"].shift(1)
# We need define the Open Long signal (RSI yersteday<55 and RSI today>55)
data_set.loc[(data_set["rsi"] > neutral_buy) & (data_set["yersteday_rsi"] < neutral_buy), "signal_long"] = 1

# We need define the Close Long signal (RSI yersteday>55 and RSI today<55) False signal
data_set.loc[(data_set["rsi"] < neutral_buy) & (data_set["yersteday_rsi"] > neutral_buy), "signal_long"] = 0

# We need define the Close Long signal (RSI yersteday>70 and RSI today<70) Over buy signal
data_set.loc[(data_set["rsi"] < overbuy) & (data_set["yersteday_rsi"] > overbuy), "signal_long"] = 0


In [ ]:
# We plot all the signla to be sure that they be correct

# Select all signal in a index list to plot only this points
idx_open = data_set.loc[data_set["signal_long"] == 1].loc["2010"].index
idx_close = data_set.loc[data_set["signal_long"] == 0].loc["2010"].index


# Adapt the size of the graph
plt.figure(figsize=(15,6))

# Plot the points of the open long signal in green
plt.scatter(data_set.loc[idx_open]["rsi"].index, data_set.loc[idx_open]["rsi"].loc["2010"], color="#57CE95", marker="^")

# Plot the points of the close long signal in blue
plt.scatter(data_set.loc[idx_close]["rsi"].index, data_set.loc[idx_close]["rsi"].loc["2010"], color="#669FEE", marker="o")

# Plot the rsi to be sure that the conditions are completed
plt.plot(data_set["rsi"].loc["2010"].index, data_set["rsi"].loc["2010"], alpha=0.35)

# Show the graph
plt.show() 

In [ ]:
# We define when the strategy must take a long buy or a short sell

# ******************************************************************************
# We need define the Open Short signal and Close Short signal

# We put threshold
oversell = 30
neutral_buy = 45

# Put nan values for the signal short columns
data_set["signal_short"] = np.nan
data_set["yersteday_rsi"] = data_set["rsi"].shift(1)
# We need define the Open short signal (RSI yersteday>45 and RSI today<45)
data_set.loc[(data_set["rsi"] < neutral_buy) & (data_set["yersteday_rsi"] > neutral_buy), "signal_short"] = -1

# We need define the Close short signal (RSI yersteday<45 and RSI today>45) False signal
data_set.loc[(data_set["rsi"] > neutral_buy) & (data_set["yersteday_rsi"] < neutral_buy), "signal_short"] = 0

# We need define the Close short signal (RSI yersteday<30 and RSI today>30) Over sell signal
data_set.loc[(data_set["rsi"] > oversell) & (data_set["yersteday_rsi"] < oversell), "signal_short"] = 0

In [ ]:
# We plot all the signal to be sure that they be correct

idx_open = data_set.loc[data_set["signal_short"] == -1].loc["2010"].index
idx_close = data_set.loc[data_set["signal_short"] == 0].loc["2010"].index

# Adapt the size of the graph
plt.figure(figsize=(15,6))

# Plot the points of the open short signal in red
plt.scatter(data_set.loc[idx_open]["rsi"].index, data_set.loc[idx_open]["rsi"].loc["2010"], color="#CE5757", marker="v")

# Plot the points of the close short signal in blue
plt.scatter(data_set.loc[idx_close]["rsi"].index, data_set.loc[idx_close]["rsi"].loc["2010"], color="#669FEE", marker="o")

# Plot the rsi to be sure that the conditions are completed
plt.plot(data_set["rsi"].loc["2010"].index, data_set["rsi"].loc["2010"], alpha=0.35)

# Show the graph
plt.show()

In [ ]:
data_set["Position"] = (data_set["signal_short"].fillna(method="ffill") + data_set["signal_long"].fillna(method="ffill"))

In [ ]:
# We plot all the signal to be sure that they be correct
year = "2010"
idx_long = data_set.loc[data_set["Position"] == 1].loc[year].index
idx_short = data_set.loc[data_set["Position"] == -1].loc[year].index

# Adapt the size of the graph
plt.figure(figsize=(15,6))

# Plot the points of the short signal in red
plt.scatter(data_set.loc[idx_short]["Adj Close"].index, data_set.loc[idx_short]["Adj Close"].loc[year], color="#CE5757", marker="v")

# Plot the points of the long signal in green
plt.scatter(data_set.loc[idx_long]["Adj Close"].index, data_set.loc[idx_long]["Adj Close"].loc[year], color="#57CE95", marker="^")

# Plot the rsi to be sure that the conditions are completed
plt.plot(data_set["Adj Close"].loc[year].index, data_set["Adj Close"].loc[year], alpha=0.35)

# Show the graph
plt.show()

In [ ]:
# Compute the percentage of variation of the asset
data_set["pct"] = data_set["Adj Close"].pct_change(1)

# Compute the return of the strategy
data_set["return"] = data_set["pct"] * (data_set["Position"].shift(1))

data_set["return"].loc["2010"].cumsum().plot(figsize=(15, 6))

# Create a function to do the RSI strategy later


In [ ]:
def rsi(val,neutral, window):
  """ 
        ------------------------------------------------------------------------------
        | Output: The function gives the returns of RSI strategy                     |
        ------------------------------------------------------------------------------
        | Inputs: -val (type dataframe pandas): Entry values of the stock            |
        |         -neutral (float): Value of neutrality, i.e. no action zone         |
        |         -window (float): rolling period for RSI                            |
        ------------------------------------------------------------------------------
  """

  # Print Error if there is no column Adj Close in the dataframe
  if "Adj Close" not in val.columns:
    ValueError("We need have a columns name Adj Close because all computation are about this column")
  
  val["rsi"] = ta.momentum.RSIIndicator(data_set["Adj Close"], window=window).rsi()


  
  """ Long buy Signal """
  # We put threshold
  overbuy = 70
  neutral_buy = 50 + neutral

  # Put nan values for the signal long columns
  val["signal_long"] = np.nan
  val["yersteday_rsi"] = data_set["rsi"].shift(1)
  # We need define the Open Long signal (RSI yersteday<55 and RSI today>55)
  val.loc[(val["rsi"]>neutral_buy) & (val["yersteday_rsi"]<neutral_buy), "signal_long"] = 1

  # We need define the Close Long signal (RSI yersteday>55 and RSI today<55) False signal
  val.loc[(val["rsi"]<neutral_buy)&(val["yersteday_rsi"]>neutral_buy), "signal_long"] = 0

  # We need define the Close Long signal (RSI yersteday>70 and RSI today<70) Over buy signal
  val.loc[(val["rsi"]<overbuy)&(val["yersteday_rsi"]>overbuy), "signal_long"] = 0



  """Short sell signal """
  # We put threshold
  oversell = 30
  neutral_buy = 50 - neutral

  # Put nan values for the signal short columns
  val["signal_short"] = np.nan
  val["yesterday_rsi"] = val["rsi"].shift(1)
  # We need define the Open Short signal (RSI yersteday>45 and RSI today<45)
  val.loc[(val["rsi"]<neutral_buy) & (val["yersteday_rsi"]>neutral_buy), "signal_short"] = -1

  # We need define the Close Short signal (RSI yersteday<45 and RSI today>45) False signal
  val.loc[(val["rsi"]>neutral_buy)&(val["yersteday_rsi"]<neutral_buy), "signal_short"] = 0

  # We need define the Close Short signal (RSI yersteday<30 and RSI today>30) Over sell signal
  val.loc[(val["rsi"]>oversell)&(val["yersteday_rsi"]<oversell), "signal_short"] = 0


  """ Compute the returns """
  # Compute the percentage of variation of the asset
  val["pct"] = val["Adj Close"].pct_change(1)

  # Compute the positions
  val["Position"] = (val["signal_short"].fillna(method="ffill")+ val["signal_long"].fillna(method="ffill"))

  # Compute the return of the strategy
  val["return"] = val["pct"]*(val["Position"].shift(1))

  return val["return"]

In [ ]:
rsi(data_set.loc["2010"], 5, 14).cumsum().plot(figsize=(15, 6))